# **SI Model on Toy Graph**

In [1]:
import networkx as nx, random

# Toy line graph
G = nx.path_graph(10)  # nodes 0-3 in a line
beta = 0.5
random.seed(1)

def simulate_SI(G, seeds, steps=12, beta=0.5):
    infected = set(seeds)
    history = [len(infected)]
    for _ in range(steps):
        new_inf = set(infected)
        for u in infected:
            for v in G.neighbors(u):
                if v not in infected and random.random() < beta:
                    new_inf.add(v)
        infected = new_inf
        history.append(len(infected))
    return history

print(simulate_SI(G, [0], steps=12, beta=beta))


[1, 2, 2, 2, 3, 4, 5, 5, 5, 6, 7, 7, 8]


# **SIR Model on Toy Graph**

In [2]:
import networkx as nx, random

# Toy line graph
G = nx.path_graph(4)  # nodes 0-3 in a line
beta = 0.5   # infection probability
gamma = 0.3  # recovery probability
random.seed(1)

def simulate_SIR(G, seeds, steps=5, beta=0.5, gamma=0.3):
    # Initialize sets
    S = set(G.nodes())
    I = set(seeds)
    R = set()
    S -= I  # remove initial infected from susceptible

    history = [(len(S), len(I), len(R))]  # track counts (S,I,R)

    for _ in range(steps):
        newI = set(I)
        newR = set(R)

        # Infection attempts
        for u in list(I):
            for v in G.neighbors(u):
                if v in S and random.random() < beta:
                    newI.add(v)
                    S.discard(v)

        # Recoveries
        for u in list(I):
            if random.random() < gamma:
                newI.discard(u)
                newR.add(u)

        I, R = newI, newR
        history.append((len(S), len(I), len(R)))

    return history

# Run the simulation: seed node 0 infected initially
print(simulate_SIR(G, [0], steps=5, beta=beta, gamma=gamma))


[(3, 1, 0), (2, 2, 0), (2, 1, 1), (1, 2, 1), (1, 0, 3), (1, 0, 3)]


# **Threshold Model**

In [3]:
import networkx as nx

# Toy line graph: 0—1—2—3—4
G = nx.path_graph(5)  # nodes 0..4 in a line

# Linear-threshold simulator (fraction rule):
# adopt if (active_neighbors / degree) >= theta
def simulate_threshold(G, seeds, theta=0.5, max_iters=10):
    A = set(seeds)               # active/adopted
    history = [len(A)]           # track cumulative adopters over time
    for _ in range(max_iters):
        to_add = []
        for u in G.nodes():
            if u in A:
                continue
            nbrs = list(G.neighbors(u))
            if not nbrs:
                continue
            frac = sum(1 for v in nbrs if v in A) / len(nbrs)
            if frac >= theta:
                to_add.append(u)
        if not to_add:
            break
        A.update(to_add)
        history.append(len(A))
    return history, A

# Seeds & threshold:
# Start with nodes 0 and 1 active; with theta=0.5 each interior node adopts
# once half of its neighbors are active → cascade down the line.
seeds = [0, 1]
theta = 0.5

hist, final_A = simulate_threshold(G, seeds, theta=theta, max_iters=10)
print("History (cumulative active each iteration):", hist)
print("Final active set:", sorted(final_A))


History (cumulative active each iteration): [2, 3, 4, 5]
Final active set: [0, 1, 2, 3, 4]
